Bulk of content is from [this](https://dbader.org/blog/python-dunder-methods) tutorial

**How are dunder methods parsed by the runtime?**

We illustrate the importance of dunder (double under) methods using an Account class. First is defining a dunder method for **Object Initialisation**: `__init__`

In [1]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []

**Object Representation:** 
* `__str__`: For the end-user, the "informal" or nicely printable string representation of an object
* `__repr__`: The "official" to-string method of an object.

Below, we also access the class name `Account` programatically instead of hardcoding it into the to-string methods

In [2]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)

In [4]:
acc = Account("bob", 10)

In [5]:
# __repr__ output:
acc

'Account'('bob', 10)

In [6]:
# __str__ output:
print(acc)

Account of bob with starting amount: 10


**Object Iteration:**
* `__len__`
* `__getitem__`
* `__reversed__`

Before we iterate on our account object, we first need to add transactions to the account. So we define a simple method `add_transaction` to add transactions, then a property `balance` to access the account balance before/after those transactions are processed

In [7]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def add_transaction(self, amount):
        """each transaction has to go into __transactions attribute"""
        if not isinstance(amount, int):
            raise ValueError('Please use int for amount')
        self._transactions.append(amount)
    
    @property
    def balance(self):
        """return the current account balance"""
        return self.amount + sum(self._transactions)

In [19]:
# we try deposit and withdraw, then get balance from account
acc = Account('Bob', 10)

acc.add_transaction(20)
acc.add_transaction(-10)
acc.add_transaction(50)
acc.add_transaction(-10)
acc.add_transaction(30)

acc.balance

90

Now that we have transactions, what if i want to know:
1. How many transactions were there?
2. List all of the transactions
3. Get the nth transaction (i.e. 1st, 3rd, so forth)

Dunder methods to the rescue! (Lack of them --> `TypeError`). The appropriate dunder methods to make the 3 above possible are:
1. `__len__`
2. `__getitem__`
3. `__getitem__`

`__reversed__` is for reversing the transaction list

In [9]:
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    # MORE Dunder methods
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]
    
    def __reversed__(self):
        return self._transactions[::-1]
    
    def add_transaction(self, amount):
        """each transaction has to go into __transactions attribute"""
        if not isinstance(amount, int):
            raise ValueError('Please use int for amount')
        self._transactions.append(amount)
    
    @property
    def balance(self):
        return self.amount + sum(self._transactions)    

In [11]:
# how many transactions:
len(acc)

5

In [12]:
# list all transactions
for transaction in acc:
    print(transaction)

20
-10
50
-10
30


In [13]:
# get a specific transaction
acc[4]

30

In [14]:
# get all transactions, reversed
reversed(acc)

[30, -10, 50, -10, 20]

In [15]:
# The dir() function returns all properties and methods of the specified object, without the values
dir(acc)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_transactions',
 'add_transaction',
 'amount',
 'balance',
 'owner']

**Object Comparison:**
* Why does > work equally well on integers, strings and other objects (as long as they are the same type)? This polymorphic behavior is possible because these objects implement one or more comparison dunder methods.
* To not have to implement all of the comparison dunder methods, we use the `functools.total_ordering` decorator which allows me to take a shortcut, only implementing `__eq__`, `__lt__` and `__gt__`:

In [16]:
from functools import total_ordering

In [18]:
@total_ordering
class Account:
    """A simple account class"""

    def __init__(self, owner, amount=0):
        """
        This is the constructor that lets us create
        objects from this class
        """
        self.owner = owner
        self.amount = amount
        self._transactions = []
    
    def __repr__(self):
        return '{!r}({!r}, {!r})'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    def __str__(self):
        return '{} of {} with starting amount: {}'.format(self.__class__.__name__,
                                         self.owner, self.amount)
    
    # ITERATION Dunder methods
    def __len__(self):
        return len(self._transactions)
    
    def __getitem__(self, position):
        return self._transactions[position]
    
    def __reversed__(self):
        return self._transactions[::-1]
    
    # ORDERING dunder methods
    def __eq__(self, other):
        return self.balance == other.balance
    
    def __lt__(self, other):
        return self.balance < other.balance
    
    def __gt__(self, other):
        return self.balance > other.balance
    
    def add_transaction(self, amount):
        """each transaction has to go into __transactions attribute"""
        if not isinstance(amount, int):
            raise ValueError('Please use int for amount')
        self._transactions.append(amount)
    
    @property
    def balance(self):
        return self.amount + sum(self._transactions)

In [21]:
# create another account to compare to previous acc
acc_2 = Account("bob", 100)

In [24]:
print(acc < acc_2)
print(acc > acc_2)
print(acc == acc_2)

True
False
False
